In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd

# Inisialisasi Firebase Admin SDK
cred = credentials.Certificate("key.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

# Fungsi untuk mengambil data dari Firebase
def load_data_from_db():
    forums_ref = db.collection('topic')
    docs = forums_ref.stream()

    # Konversi data Firebase ke DataFrame
    forums_data = []
    for doc in docs:
        forum = doc.to_dict()
        forums_data.append(forum)

    df = pd.DataFrame(forums_data)
    return df


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Load data dari Firebase
df = load_data_from_db()

# Vectorizer dan Similarity
cv = CountVectorizer(max_features=5000, stop_words='english')
df['tags'] = df['tags'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
vector = cv.fit_transform(df['tags']).toarray()
similarity = cosine_similarity(vector)


# Simpan model ke Pickle
with open('model.pkl', 'wb') as file:
    pickle.dump((df, cv, similarity), file)